In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:


import numpy as np

import tensorflow as tf
from tensorflow import keras

tf.__version__



'2.15.0'

In [3]:


from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.applications.xception import preprocess_input



In [5]:
model = keras.models.load_model('/content/drive/MyDrive/Colab Notebooks/ZoomCamp/Projects/Capstone 1/Models/SelectedModel.h5')

img = load_img('/content/drive/MyDrive/Colab Notebooks/ZoomCamp/Projects/Capstone 1/ISIC-images/val/melanoma/ISIC_0015243.jpg', target_size=(400, 533))

x = np.array(img)
X = np.array([x])

X = preprocess_input(X)

preds = model.predict(X)

preds



1/1 [==============================] - 2s 2s/step


array([[ 2.569057 ,  1.3068167, -3.2380097]], dtype=float32)

In [6]:
classes = [
    'Melanoma',
    'Nevus',
    'Seborrheic_Keratosis',
    ]

dict(zip(classes, preds[0]))



{'Melanoma': 2.569057, 'Nevus': 1.3068167, 'Seborrheic_Keratosis': -3.2380097}

In [9]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)

tflite_model = converter.convert()

with open('SCP-model.tflite', 'wb') as f_out:
    f_out.write(tflite_model)

In [10]:
!ls -lh

total 161M
-rw-r--r-- 1 root root  81M Dec 18 21:08 clothing-model.tflite
drwx------ 6 root root 4.0K Dec 18 20:58 drive
drwxr-xr-x 1 root root 4.0K Dec 15 14:24 sample_data
-rw-r--r-- 1 root root  81M Dec 18 21:10 SCP-model.tflite


In [11]:
import tensorflow.lite as tflite

interpreter = tflite.Interpreter(model_path='SCP-model.tflite')
interpreter.allocate_tensors()

input_index = interpreter.get_input_details()[0]['index']
output_index = interpreter.get_output_details()[0]['index']

interpreter.set_tensor(input_index, X)
interpreter.invoke()
preds = interpreter.get_tensor(output_index)

classes = [
    'dress',
    'hat',
    'longsleeve',
    'outwear',
    'pants',
    'shirt',
    'shoes',
    'shorts',
    'skirt',
    't-shirt'
]

dict(zip(classes, preds[0]))

{'dress': 2.5690563, 'hat': 1.3068198, 'longsleeve': -3.2380116}

Remove Dependency to TF

In [12]:
!pip install keras-image-helper
!pip install --extra-index-url https://google-coral.github.io/py-repo/ tflite_runtime

Looking in indexes: https://pypi.org/simple, https://google-coral.github.io/py-repo/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 9.5 MB/s eta 0:00:00


In [ ]:
import tflite_runtime.interpreter as tflite
from keras_image_helper import create_preprocessor

interpreter = tflite.Interpreter(model_path='SCP-model.tflite')
interpreter.allocate_tensors()

input_index = interpreter.get_input_details()[0]['index']
output_index = interpreter.get_output_details()[0]['index']

preprocessor = create_preprocessor('xception', target_size=(299, 299))

url = 'http://bit.ly/mlbookcamp-pants'
X = preprocessor.from_url(url)

interpreter.set_tensor(input_index, X)
interpreter.invoke()
preds = interpreter.get_tensor(output_index)

classes = [
    'dress',
    'hat',
    'longsleeve',
    'outwear',
    'pants',
    'shirt',
    'shoes',
    'shorts',
    'skirt',
    't-shirt'
]

dict(zip(classes, preds[0]))

